# Quantum Convolutional Neural Network

This is an implementation of 8 Qubit QCNN circuit testing various ansatze.

In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)

# QCNN Circuit

Here are various unitary ansatze to test

In [33]:
def U_TTN(theta1, theta2, wires):
    qml.RY(theta1, wires = wires[0])
    qml.RY(theta2, wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_13(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRZ(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])

def U_15(params, wires): # 4 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[1], wires[0]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])
    
def V_0(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

def V_1(theta, wires):
    qml.CRX(theta, wires = [wires[0], wires[1]])

def F(wires):
    qml.CNOT(wires = [wires[0], wires[1]])

Define general layers that will be used in the circuits.

In [21]:
# Convolution Layer1
def conv_layer1(U, params):
    U(params, wires = [0,7])
    for i in range (0,8,2):
        U(params, wires = [i, i+1])
    for i in range (1,7,2):
        U(params, wires = [i, i+1])
    
def conv_layer2(U, params):
    U(params, wires = [0,2])
    U(params, wires = [4,6])
    U(params, wires = [2,4])
    U(params, wires = [0,6])
    
def pooling_layer1(V_0, V_1, params):
    for i in range(0,8,2):
        V_0(params[0], wires = [i+1, i])
    for i in range(0,8,2):
        qml.PauliX(wires = i+1)
    for i in range(0,8,2):
        V_1(params[1], wires = [i+1, i])
        

def pooling_layer2(V_0, V_1, params): # 2params
    V_0(params[0], wires = [2,0])
    V_0(params[0], wires = [6,4])
    
    qml.PauliX(wires = 2)
    qml.PauliX(wires = 6)
    
    V_1(params[1], wires = [2,0])
    V_1(params[1], wires = [6,4])

    

Here we define various possible embedding methods.

In [4]:
def data_embedding(X, embedding_type = 'Amplitude'):
    if embedding_type == 'Amplitude':
        AmplitudeEmbedding(X, wires = range(8), normalize = True)
    elif embedding_type == 'Angle':
        AngleEmbedding(X, wires = range(8))

In [42]:
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding


dev = qml.device('default.qubit', wires = 8)
@qml.qnode(dev)
def QCNN(X, params, embedding_type = 'Amplitude'):
    
    #param1, param2, param3, param4 = params[0], params[1], params[2], params[3]
    param1 = [params[0], params[1], params[2], params[3]]
    param2 = [params[4], params[5]]
    param3 = [params[6], params[7], params[8], params[9]]
    param4 = [params[10], params[11]]
    
    data_embedding(X, embedding_type = embedding_type)
    
    conv_layer1(U_15, param1)
    pooling_layer1(V_0, V_1, param2)
    conv_layer2(U_15, param3)
    pooling_layer2(V_0, V_1, param4)
    
    F(wires = [0,4])
    return qml.expval(qml.PauliZ(4))

# Training Quantum Circuits

In [6]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def cost(params, X, Y):
    predictions = [QCNN(x, params) for x in X]
    return square_loss(Y, predictions)

def accuracy(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 0.5:
            acc = acc + 1
    return acc / len(labels)

In [36]:
def circuit_training(X_train, Y_train):
    #params = [np.random.randn(4), np.random.randn(2), np.random.randn(4), np.random.randn(2)]
    params = np.random.randn(12)
    steps = 200
    learning_rate = 0.1
    batch_size =25
    opt = qml.NesterovMomentumOptimizer(learning_rate)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch), params)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return params

### Results

In [8]:
import tensorflow as tf
def data_load_and_process(classes = [0,1]):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0
    
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
    
    
    x_train_01 = x_train[x_train_filter_01]
    y_train_01 = y_train[x_train_filter_01]
    x_test_01 = x_test[x_test_filter_01]
    y_test_01 = y_test[x_test_filter_01]
    
    
    
    x_train= tf.cast(x_train_01, tf.float32)
    x_test=tf.cast(x_test_01, tf.float32)
    
    x_train = tf.image.resize(x_train[:], (256, 1)).numpy()
    x_test = tf.image.resize(x_test[:], (256, 1)).numpy()
    
    x_train, x_test = tf.squeeze(x_train), tf.squeeze(x_test)
    
    return x_train, x_test, y_train_01, y_test_01

In [12]:
x_train, x_test, y_train, y_test = data_load_and_process()
#y_train_2, y_test_2 = np.array(y_train, dtype = "int"), np.array(y_test, dtype = "int")
#y_train_2, y_test_2 = y_train_2 * 2 - 1, y_test_2 * 2 - 1

In [38]:
print(x_train.shape, y_train.shape)
params = [np.random.randn(4), np.random.randn(2), np.random.randn(4), np.random.randn(2)]

(12665, 256) (12665,)


In [39]:
from sklearn import decomposition

pca = decomposition.PCA(n_components = 8)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

Training QCNN with amplitdue encoding.    

In [43]:
trained_params_amp = circuit_training(x_train, y_train)

iteration:  0  cost:  0.6516509151394178
iteration:  10  cost:  0.11551695476619508
iteration:  20  cost:  0.08705409549652122
iteration:  30  cost:  0.03230764607467249
iteration:  40  cost:  0.038195760856150715
iteration:  50  cost:  0.05271543359043137
iteration:  60  cost:  0.051772828004233974
iteration:  70  cost:  0.06148899723118883
iteration:  80  cost:  0.039396990852718694
iteration:  90  cost:  0.03241109224240893
iteration:  100  cost:  0.046324785743376534
iteration:  110  cost:  0.045720752359008125
iteration:  120  cost:  0.046850186882593396
iteration:  130  cost:  0.029606568119009618
iteration:  140  cost:  0.026877343178648684
iteration:  150  cost:  0.05790818609053103
iteration:  160  cost:  0.03305564810709723
iteration:  170  cost:  0.043388537981140146
iteration:  180  cost:  0.054404610460382495
iteration:  190  cost:  0.04176706345731735


In [44]:
predictions = [QCNN(x, trained_params_amp) for x in x_test]
accuracy(predictions, y_test)

0.9881796690307328